## This notebook was needed to clean the data and export it to csvbecause the kernel keeps dying with a huge dataset and using while R as an extension.

In [2]:
import pandas as pd
import re

In [3]:
#read in the csv files into df
df18 = pd.read_csv('/Users/dillonquan/Desktop/ds_jobs/alldata.csv')
df19 = pd.read_csv('indeed2019All.csv', encoding = 'latin-1')

In [4]:
#run this to make sure there are no duplicates within the data when sampling
df = df19.groupby(['position', 'company','location']).sum().reset_index().sort_values(by = 'flag', ascending = False)
df19 = ''
#create a state column
df['state'] = df['location'].str[-2:]

In [5]:
def categorize(s):
    patterns = ['.*data sci.*','.*data.*eng.*', '.*analy.*', '.*machine.*']
    for idx, pattern in enumerate(patterns):
        reg = re.compile(pattern)
        if reg.search(s):
            return idx
    return -1

### 2019 Indeed Data. Randomly sampled 7000 due to the amount of data we scraped

In [5]:
#random sample 7000 job postings and setting the big dataframe to be nothing
df_ran = df.sample(n = 7000, random_state = 1)

In [6]:
#normalizing the text
df_ran['position'] = df_ran['position'].apply(lambda x : x.lower())
df_ran = df_ran.reset_index(drop = True)
df_ran.head()

,position,company,location,flag,state
0,quantitative researcher,MKT MEDIASTATS,"Cambridge, MA",5,MA
1,distinguished data scientist,Verizon,"Irving, TX",5,TX
2,senior data analytics analyst,Allstate,"Chicago, IL",1,IL
3,senior organizational change consultant,itelligence,"Chicago, IL",1,IL
4,adobe analytics analyst,Turbine Labs,"Greenwood Village, CO",1,CO


In [7]:
#calling categorize function
df_ran['label'] = -1
for idx, row in df_ran.iterrows():
    cat = categorize(row.position)
    if cat != -1:
        df_ran.loc[idx, 'label'] = cat

In [8]:
d19 = df_ran.groupby('label').count().reset_index()

### 2018 Indeed Data

In [6]:
#normalizing the positions for 2018 dataset
df18['position'] = df18['position'].apply(lambda x : str(x).lower())
df18 = df18.reset_index(drop = True)
df18.head()

,position,company,description,reviews,location
0,development director,ALS TDI,Development Director\nALS Therapy Development ...,NaN,"Atlanta, GA 30301"
1,an ostentatiously-excitable principal research...,The Hexagon Lavish,"Job Description\n\n""The road that leads to acc...",NaN,"Atlanta, GA"
2,data scientist,Xpert Staffing,"Growing company located in the Atlanta, GA are...",NaN,"Atlanta, GA"
3,data analyst,Operation HOPE,DEPARTMENT: Program OperationsPOSITION LOCATIO...,44.0,"Atlanta, GA 30303"
4,assistant professor -tt - signal processing & ...,Emory University,DESCRIPTION\nThe Emory University Department o...,550.0,"Atlanta, GA"


In [7]:
#categorizing the positions
df18['label'] = -1
for idx, row in df18.iterrows():
    cat = categorize(row.position)
    if cat != -1:
        df18.loc[idx, 'label'] = cat

In [9]:
#creating a csv for wordcloud
df18.to_csv('pos2018.csv')

In [18]:
d18 = df18.groupby('label').count().reset_index()

#only taking the jobs that are in SEAM
t1 = d19[['label', 'position']].loc[(d19.label >= 0),:]
t2 = d18[['label', 'position']].loc[(d18.label >= 0),:]

#creating the csv files to create slope graphs
t1.to_csv('2019_count.csv')
t2.to_csv('2018_count.csv')